## Sentinel-2 cloud/shadow-free composite

This example creates and downloads a large cloud/shadow free composite from a year of Sentinel-2 SR imagery.  The area of interest is over Accra, Ghana.  It demonstrates approaches for staying within Earth Engine computational and memory limits, and for compositing heavily clouded areas. 

CLI commands equivalent to the API code snippets are given in the comments where possible.

### Setup

`geemap` is required to run the notebook.  You can uncomment the cell below to install it, if it isn't installed already.

In [1]:
# geemap should be installed if it isn't already.
# import sys
# if 'conda' in sys.prefix:
#     # install into the conda environment the notebook is being run from
#     !conda install --yes --prefix {sys.prefix} -c conda-forge geemap
# else:
#     # install into the python environment the notebook is being run from
#     !{sys.executable} -m pip install geemap

In [2]:
import logging
import ee
import geedim as gd
import geemap.foliumap as geemap

# initialise earth engine with the high-volume endpoint
gd.Initialize()

### Create and search a Sentinel-2 SR collection

In this step we search for images to composite, over the area and time period of interest.  

Searching and compositing Sentinel-2 collections over large areas or time spans can be time-consuming, in part due to the resolution of the imagery; but also due to the computational burden of the `geedim` cloud/shadow masking algorithms.  To reduce the number of images requiring cloud/shadow masking in the search, we specify a `custom_filter` that places an upper limit on the already existing `CLOUDY_PIXEL_PERCENTAGE` [image granule property](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#image-properties).  This `custom_filter` is fast to apply and helps speed up the search by reducing the number of images requiring `geedim` cloud/shadow masking.  It is also recommended to keep the search region and time span to a reasonable minimum.

While the `custom_filter` applies to properties of the entire image granule, the `fill_portion` and `cloudless_portion` filters apply to `geedim` calculated portions of the search region.   Here they are used to limit the search results to relatively large and cloud/shadow-free image portions.  

We specify an aggresive [cloud probability threshold](../api.rst#MaskedImage) of 40% (`prob=40`), and a large [cloud/shadow dilation distance](../api.rst#MaskedImage) of 100m (`buffer=100`), to avoid the inclusion of cloudy pixels in the resulting images.  

In [3]:
# geojson search polygon
region = {
    'type': 'Polygon', 'coordinates': [[
    [-0.37, 5.75], [-0.37, 5.5], [0.0, 5.5], [0.0, 5.75], [-0.37, 5.75]
  ]],
}

# create and search the Sentinel-2 TOA collection
coll = gd.MaskedCollection.from_name('COPERNICUS/S2_SR_HARMONIZED')
filt_coll = coll.search(
    start_date='2019-01-01', end_date='2020-01-01', region=region, 
    cloudless_portion=75, fill_portion=30, 
    custom_filter='CLOUDY_PIXEL_PERCENTAGE<25', prob=40, buffer=100,
)

# print the search results
print(f'Found {len(filt_coll.properties)} images.')
print(f'Image property descriptions:\n\n{filt_coll.schema_table}\n')
print(f'Search Results:\n\n{filt_coll.properties_table}')

# !geedim config --prob 40 --buffer 100 search -c s2-sr-hm -s 2019-01-01 -e 2020-01-01 --bbox -0.37 5.5 -0.0 5.75 -cf "CLOUDY_PIXEL_PERCENTAGE<25" -cp 75 -fp 30

Found 26 images.
Image property descriptions:

ABBREV     NAME                             DESCRIPTION
---------  -------------------------------  ----------------------------------------------
ID         system:id                        Earth Engine image id
DATE       system:time_start                Image capture date/time (UTC)
FILL       FILL_PORTION                     Portion of region pixels that are valid (%)
CLOUDLESS  CLOUDLESS_PORTION                Portion of filled pixels that are cloud/shadow
                                            free (%)
RADQ       RADIOMETRIC_QUALITY              Radiometric quality check
GEOMQ      GEOMETRIC_QUALITY                Geometric quality check
SAA        MEAN_SOLAR_AZIMUTH_ANGLE         Solar azimuth angle (deg)
SZA        MEAN_SOLAR_ZENITH_ANGLE          Solar zenith angle (deg)
VAA        MEAN_INCIDENCE_AZIMUTH_ANGLE_B1  View (B1) azimuth angle (deg)
VZA        MEAN_INCIDENCE_ZENITH_ANGLE_B1   View (B1) zenith angle (deg)

Search Re

#### Create composite image

Next we create a [*medoid*](../api.rst#compositemethod) composite image from the search result images.  Again we specify a [cloud probability threshold](../api.rst#MaskedImage) of 40% (`prob=40`), and a [cloud/shadow dilation distance](../api.rst#MaskedImage) of 100m (`buffer=100`), to aggressively mask cloudy pixels.  These parameters result in some false positive cloud/shadow detections (i.e. mask some valid pixels), but with numerous component images covering the same area, there are enough valid pixels to produce a full coverage composite.

Of the [available compositing algorithms](../api.rst#compositemethod), the *medoid* method is the most memory and computation intensive.  Computation scales roughly quadratically with the number of component images, so it is important to keep those to a necessary minimum.  The number of images returned by the search (i.e. 26) is low enough to produce a reasonably fast composite in this case.  Limiting the components to mostly cloud/shadow-free images, and aggressively masking these, helps improve the composite quality. 

In [4]:
medoid_im = filt_coll.composite('medoid', prob=40, buffer=100)

### Visualise component and composite images

Now we use `geemap` to display the first 4 component images, their cloud/shadow masks, and the *medoid* composite.  You can use the layer button in the top right hand corner to turn on/off the image layers.  The composite will take a few minutes to load.  
 
<div class="alert alert-info">

Note

You need to download and run the notebook to view the images.

</div>

In [5]:
# geemap visualisation parameters
s2_vis_params = dict(min=0, max=4000, bands=['B4', 'B3', 'B2'], gamma=1.5)
mask_vis_params = dict(min=0, max=1)

map = geemap.Map()
map.centerObject(ee.Geometry(region), 11)

# iterate over the first 4 component image ID's
for im_id in list(filt_coll.properties.keys())[:4]:
    # create image and mask, and add to map
    im = gd.MaskedImage.from_id(im_id, prob=40, buffer=100)
    vis_image = im.ee_image.clip(region)
    cloudless_mask = vis_image.select('CLOUDLESS_MASK')
    
    map.addLayer(vis_image, s2_vis_params, f'Image {im_id[-38:]}')
    map.addLayer(cloudless_mask, mask_vis_params, f'Cl mask {im_id[-38:]}')

# add the composite to the map
map.addLayer(
    medoid_im.ee_image.clip(region), s2_vis_params, 'Medoid composite'
)

region_im = ee.Image().byte().paint(
    featureCollection=ee.Geometry(region), width=2, color=1
)
map.addLayer(region_im, dict(palette=['FF0000']), 'Region')

map

### Download

Here we download the composite image directly.  `crs`, `scale` and `region` parameters must be specified, as the composite has no fixed projection.  Computation of the *medoid* composite is memory intensive, so we specify a `max_tile_size` of 16MB.  Without this, the download would exceed the [user memory limit](../api.rst#computed-images-and-user-memory).  

In [6]:
medoid_im.download(
    's2_medoid_im.tif', region=region, crs='EPSG:32735', scale=10, 
    dtype='uint16', max_tile_size=16, overwrite=True,
)

# !geedim config --prob 40 --buffer 100 search -c s2-sr-hm -s 2019-01-01 -e 2020-01-01 --bbox -0.37 5.5 -0.0 5.75 -cf "CLOUDY_PIXEL_PERCENTAGE<25" -cp 75 -fp 30 composite -cm medoid download --crs EPSG:32735 --scale 10 --dtype uint16 --max-tile-size 16 -o

s2_medoid_im.tif: |                                                                                     | 0.00…

### Export

Lastly we demonstrate an alternative to the [direct download approach](#Download) i.e. we export the composite image to an Earth Engine asset, and then download the computed asset image.  Exporting is not subject to the user memory limit, and once computed, the asset image can be downloaded without the need for reducing `max_tile_size`.  Bear in mind that exporting is usually slower than downloading, and exported GeoTIFFs can not be populated with [STAC metadata](../api.rst#image-metadata). 

In [8]:
# 'geedim' = the name of a valid Earth Engine asset project
medoid_asset_id = f'projects/geedim/assets/s2_medoid_im'

# export to earth engine asset
medoid_task = medoid_im.export(
    medoid_asset_id, type='asset', region=region, crs='EPSG:32735', 
    scale=10, dtype='uint16', wait=True,
)
# !geedim config --prob 40 --buffer 100 search -c s2-sr-hm -s 2019-01-01 -e 2020-01-01 --bbox -0.37 5.5 -0.0 5.75 -cf "CLOUDY_PIXEL_PERCENTAGE<25" -cp 75 -fp 30 composite -cm medoid export -t asset -f geedim --crs EPSG:32735 --scale 10 --dtype uint16

# create a MaskedImage from the earth engine asset
medoid_asset_im = gd.MaskedImage.from_id(medoid_asset_id)
# download asset image
medoid_asset_im.download('s2_medoid_im.tif', overwrite=True)

# !geedim download -i projects/geedim/assets/s2_medoid_im -o

Preparing projects-geedim-assets-s2_medoid_im: done 


Exporting projects-geedim-assets-s2_medoid_im: |                                                              …

s2_medoid_im.tif: |                                                                                     | 0.00…

There is no STAC entry for: projects/geedim/assets/s2_medoid_im
